<a href="https://colab.research.google.com/github/venkate5hgunda/MachineLearningProjectsHome/blob/master/CSE598_Assignment1_Part_a1b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# copied the imports from Assignment1:a1a.py
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import time
import datetime
import utils

In [2]:
# Define paramaters for the model
learning_rate = 0.1
batch_size = 200 # 10000
n_epochs = 30
n_steps_per_epoch = 32 # used to be larger, but taking forever for an epoch
n_train = 60000
n_test = 10000

In [3]:
# Step 1: Read in data
mnist_folder = 'data/mnist'
if os.path.isdir(mnist_folder) != True:
    os.mkdir('data')
    os.mkdir(mnist_folder)
utils.download_mnist(mnist_folder)
# train, val, test = utils.read_mnist(mnist_folder, flatten=True)
train, val, test = utils.read_mnist(mnist_folder)

Successfully downloaded data/mnist/train-images-idx3-ubyte.gz
Successfully downloaded data/mnist/train-labels-idx1-ubyte.gz
Successfully downloaded data/mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded data/mnist/t10k-labels-idx1-ubyte.gz


In [4]:
# Step 2: Create datasets and iterator
# create training Dataset and batch it
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000) # if you want to shuffle your data
train_data = train_data.batch(batch_size)

x_train, x_val, y_train, y_val = train_test_split(train[0],train[1],test_size=0.2)
x_train, x_val = x_train.reshape(-1,28,28,1), x_val.reshape(-1,28,28,1)

In [5]:
# create testing Dataset and batch it
test_data = tf.data.Dataset.from_tensor_slices(test)
test_data = test_data.shuffle(1000)
test_data = test_data.batch(batch_size)

In [6]:
# Model Definition
# Reference for Model Definition: https://www.kaggle.com/cdeotte/how-to-choose-cnn-architecture-mnist?scriptVersionId=5965574&cellId=31

model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(32,kernel_size=3,activation='relu',input_shape=(28,28,1)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32,kernel_size=3,activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.4))

model.add(tf.keras.layers.Conv2D(64,kernel_size=3,activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64,kernel_size=3,activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.4))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(10,activation='softmax'))

model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [7]:
# Data Augmentation

augmented_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=10,zoom_range=0.1,width_shift_range=0.1,height_shift_range=0.1)
learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.95 ** (x+n_epochs))

# Then fit the model, and we should get our accuracy number
accuracy = model.fit_generator(augmented_data_generator.flow(x_train, y_train, batch_size=batch_size), epochs=n_epochs, steps_per_epoch=n_steps_per_epoch, validation_data=(x_val, y_val), callbacks=[learning_rate_scheduler], verbose=1)
print(accuracy)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/30
32/32 [==============================] - 37s 1s/step - loss: 2.8509 - accuracy: 0.1917 - val_loss: 2.3125 - val_accuracy: 0.0970 - lr: 2.1464e-04
Epoch 2/30
32/32 [==============================] - 35s 1s/step - loss: 1.9223 - accuracy: 0.3988 - val_loss: 2.3288 - val_accuracy: 0.1146 - lr: 2.0391e-04
Epoch 3/30
32/32 [==============================] - 35s 1s/step - loss: 1.4100 - accuracy: 0.5472 - val_loss: 2.3656 - val_accuracy: 0.1129 - lr: 1.9371e-04
Epoch 4/30
32/32 [==============================] - 35s 1s/step - loss: 1.0893 - accuracy: 0.6452 - val_loss: 2.3807 - val_accuracy: 0.1129 - lr: 1.8403e-04
Epoch 5/30
32/32 [==============================] - 35s 1s/step - loss: 0.8789 - accuracy: 0.7117 - val_loss: 2.4090 - val_accuracy: 0.1129 - lr: 1.7482e-04
Epoch 6/30
32/32 [==============================] - 35s 1s/step - loss: 0.7578 - accuracy: 0.7578 - val_loss: 2.4423 - val_accuracy: 0.1129 - lr: 1.6608e-04
Epoch 7/30
32/32 [==============================] - 35s 1s